## 동아일보

In [14]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import logging
import os
import re
from datetime import datetime
from urllib.parse import urlparse

logging.basicConfig(level='DEBUG')

header = {'User-Agent': ('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 '
                         '(KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'),}

output_dir = './donga'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

categories = {
    '정치': 'https://www.donga.com/news/Politics',
    '경제': 'https://www.donga.com/news/Economy',
    '국제': 'https://www.donga.com/news/Inter',
    '사회_사건': 'https://www.donga.com/news/Society/Event', # 앞 분기 기준
    '경제_과학': 'https://www.donga.com/news/It/List', # 앞 분기 기준
    '사회': 'https://www.donga.com/news/Society',
    '스포츠': 'https://www.donga.com/news/Sports',
}

result_categories = {
    '정치': 'politics',
    '경제': 'economy',
    '국제': 'world',
    '사회_사건': 'accident',
    '경제_과학': 'science',
    '사회': 'society',
    '스포츠': 'sports',
}

# 현재 날짜의 오전 8시 (시간대는 +09:00 기준)
today = datetime.now().replace(hour=8, minute=0, second=0, microsecond=0, tzinfo=datetime.now().astimezone().tzinfo)

def scrap_article(article_url, category):
    response = requests.get(article_url, headers=header)
    soup = BeautifulSoup(response.text, 'html.parser')

    try:
        page_category = soup.find('ol', class_='breadcrumb').find('li', class_='breadcrumb_item').find('a').get_text(strip=True)
        logging.info(f"page_category: {page_category}")
        
        logging.info(f"category: {category}")

        category_parts = category.split('_')
        
        # 카테고리가 아닌 글은 패스
        if page_category not in category_parts:
            logging.info(f"{article_url} 는 해당 카테고리가 아니어서 생략합니다. ")
            return None
    except AttributeError:
        logging.error(f"{article_url} 는 찾을 수 없습니다.")
        return None

    # 헤드라인 스크랩
    try:
        title = soup.find('section', {'class': 'head_group'}).find('h1').get_text(strip=True)
    except AttributeError:
        title = 'N/A'
        
    try:
        date_button = soup.find('button', {'aria-controls': 'dateInfo'})
        time_span = date_button.find('span', {'aria-hidden': 'true'})
        time = time_span.get_text(strip=True)  # "2024-09-30 21:44" 형태
        time = time.replace(" ", "T") + ":00+09:00"  # ISO 8601 형식으로 변환 (예: 2024-09-30T21:44:00+09:00)
    except AttributeError:
        time = 'N/A'

    # 본문 스크랩
    try:
        # 광고, 이미지 등 제외하고 텍스트만 추출
        content_section = soup.find('section', class_='news_view')
        for tag in content_section(['figure', 'script', 'div', 'ul']):  # 불필요한 태그 제거
            tag.decompose()
        content = content_section.get_text(separator="\n", strip=True)
    except AttributeError:
        content = 'N/A'

    result_category = result_categories.get(category, 'unknown')

    return {'title': title, 'source_url': article_url, 'content': content, 'category': result_category, 'source_created_at': time, 'press_id': 2, 'press_name' : '동아일보'}

# 파일 생성
def save_article_as_json(article_data, date, file_idx):
    category = article_data.get('category')
    file_name = date + "-" + str(file_idx) + '.json'
    file_path = os.path.join(output_dir + "/" + category, file_name)
    
    # 단일 dataframe 생성
    single_article_df = pd.DataFrame([article_data])
    
    # json으로 변환
    single_article_df.to_json(file_path, orient='records', lines=True, force_ascii=False)
    logging.info(f"Article saved to {file_path}")

def check_for_new_articles(base_url, category):
    response = requests.get(base_url, headers=header)
    soup = BeautifulSoup(response.text, 'html.parser')

    article_count = 0
    cards = soup.find_all('article', class_='news_card')
    if category != '사회_사건':
        cards.reverse()
    for card in cards:
        if article_count >= 4:
            break

        link_tag = card.find('a', href=True)
        if link_tag and 'href' in link_tag.attrs:
            article_link = link_tag['href']
        else:
            logging.warning("article_link is None or missing 'href'.")
            continue
    
        logging.debug(f"Scraping article: {article_link}")

        result = scrap_article(article_link, category)

        if result is None:
            logging.error("result is None");
            continue

        if result.get('category') not in ['accident', 'science']:
            # 'source_created_at'를 datetime 객체로 변환
            article_time = datetime.strptime(result.get('source_created_at'), '%Y-%m-%dT%H:%M:%S%z')
            # 오늘 오전 8시 이후가 아니면 continue
            if article_time < today:
                logging.info(f"{article_link} 은 오늘 오전 8시 이전 기사이므로 생략합니다.")
                continue
        
        print(result)
        article_count += 1
        date = re.match(r'^[^T]+', result.get('source_created_at')).group()
        save_article_as_json(result, date, article_count)

def scrap_all_categories():
    
    for category, base_url in categories.items():
        logging.info(f"category (base_url): {category} ({base_url})")
        check_for_new_articles(base_url, category)
        time.sleep(2)

scrap_all_categories()

INFO:root:category (base_url): 사회_사건 (https://www.donga.com/news/Society/Event)
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.donga.com:443
DEBUG:urllib3.connectionpool:https://www.donga.com:443 "GET /news/Society/Event HTTP/11" 200 None
DEBUG:root:Scraping article: https://www.donga.com/news/Society/article/all/20241008/130174875/1
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.donga.com:443
DEBUG:urllib3.connectionpool:https://www.donga.com:443 "GET /news/Society/article/all/20241008/130174875/1 HTTP/11" 200 None
INFO:root:page_category: 사회
INFO:root:category: 사회_사건
INFO:root:Article saved to ./donga/accident\2024-10-08-1.json
DEBUG:root:Scraping article: https://www.donga.com/news/Society/article/all/20241008/130174358/2
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.donga.com:443
DEBUG:urllib3.connectionpool:https://www.donga.com:443 "GET /news/Society/article/all/20241008/130174358/2 HTTP/11" 200 None
INFO:root:page_c

{'title': '고속도로서 택시 기사 폭행…KAIST 교수 징역 1년6개월 구형', 'source_url': 'https://www.donga.com/news/Society/article/all/20241008/130174875/1', 'content': '서울에서 대전으로 택시를 타고 이동하던 중 고속도로에서 택시 기사를 폭행한 혐의로 재판에 넘겨진 한국과학기술원(KAIST) 교수에게 검찰이 실형을 구형했다.\n대전지법 형사4단독 이제승 판사는 8일 오전 11시10분 317호 법정에서 특정 범죄 가중 처벌 등에 관한 법률 위반(운전자 폭행 등), 공무집행방해 혐의로 기소된 A(62)씨에 대한 결심 공판을 진행했다.\n검찰은 이날 “고속도로를 운전하던 피해자 B씨에게 폭행을 저질렀고 출동한 경찰도 폭행했다”며 A씨에게 징역 1년 6개월을 구형했다.\nA씨 측 변호인은 최후 변론에서 “변명의 여지가 없는 잘못을 저질러 깊이 반성하고 있고 불안감을 느꼈을 피해자에게 상당히 죄송하며 피해자가 제출한 엄벌탄원서를 보고 또다시 반성했다”며 “피고인은 수학을 전공하며 학계와 교육계에 본인의 인생을 한평생 바쳐 살아왔고 현재 직위해제와 정직 3개월이라는 중징계 처분을 받은 점을 고려해 달라”고 말했다.\nA씨는 최후 진술에서 “피해자들에게 머리 숙여 사과하며 스스로에 대한 실망감과 자괴감으로 하루하루 후회하고 반성하는 삶을 살고 있다”며 “사건 후로 음주를 멀리하고 평생 그럴 예정이며 피해자들이 입은 피해와 이 사건이 사회에 끼치는 영향을 생각하며 살겠다”고 호소했다.\n재판부는 오는 11월15일 오후 2시에 A씨에 대한 선고를 이어갈 방침이다.\nA씨는 지난해 12월30일 서울 강남에서 대전으로 이동하기 위해 택시를 타고 고속도로를 이동하던 중 택시 기사인 B씨의 뺨을 때리고 팔을 잡아끄는 등 폭행한 혐의로 재판에 넘겨졌다.\n당시 B씨가 항의했으나 A씨는 택시가 약 30㎞를 달리던 동안 폭행 및 운전 방해 등을 했으며 신고를 받아 출동한 경찰에게도 폭행을 했던 것으

INFO:root:category: 사회_사건
INFO:root:Article saved to ./donga/accident\2024-10-08-2.json
DEBUG:root:Scraping article: https://www.donga.com/news/Society/article/all/20241008/130173774/2
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.donga.com:443
DEBUG:urllib3.connectionpool:https://www.donga.com:443 "GET /news/Society/article/all/20241008/130173774/2 HTTP/11" 200 None
INFO:root:page_category: 사회


{'title': '대법, ‘김봉현 술접대’ 전·현직 검사 유죄 취지 파기환송', 'source_url': 'https://www.donga.com/news/Society/article/all/20241008/130174358/2', 'content': '대법원이 라임자산운용의 전주(錢主) 김봉현 전 스타모빌리티 회장(수감 중)으로부터 술 접대를 받은 현직 검사와 자리를 주선한 검사 출신 변호사에게 무죄를 선고한 원심판결을 파기 환송했다.\n대법원 2부(주심 대법관 오경미)는 8일 청탁금지법 위반 혐의로 재판에 넘겨진 나모 검사와 부장검사 출신 이모 변호사, 김 전 회장에게 무죄를 선고한 원심을 깨고 사건을 서울남부지법으로 돌려보냈다.\n나 검사는 2019년 7월 서울 강남구 청담동 유흥업소에서 김 전 회장으로부터 100만 원이 넘는 향응을 받은 혐의로 재판에 넘겨졌다. 이 변호사는 김 전 회장과 공모해 검사 3명에게 술 접대를 한 혐의를 받는다.\n당초 검찰은 김 전 회장이 나 변호사와 이 변호사에게 1인당 114만5333원의 술값을 썼다고 판단했다. 그러나 원심은 ‘참석자가 7명이었다’는 피고인 측 주장을 받아들였고, 향응비를 1인당 93만9167만 원으로 보고 100만 원을 초과하지 않는다며 무죄를 선고했다.\n청탁금지법은 공직자 등은 직무 관련 여부 및 기부·후원·증여 등 명목과 관계없이 동일인으로부터 1회 100만 원 또는 매 회계연도 300만 원 초과 금품을 받거나 요구 또는 약속해서는 안 된다고 정하고 있다. 이에 따라 대법원은 그간 나 검사와 이 변호사에 대한 향응 가액이 100만 원을 초과하는지 여부를 집중 살펴왔다.\n대법원은 이날 “여러 사정을 고려하여 참석자별로 접대에 들어간 비용을 객관적이고 규범적으로 판단하고, 이를 구분할 수 있다면 그에 따른 안분을 하는 것도 필요하다”며 검사의 상고를 인용해 사건을 파기 환송했다.\n대법원은 “향응 가액 산정에 관한 기존 법리가 청탁금지법 위반죄의 향응 가액 산정에서도 유지된다고 본다”면서 “3명 

INFO:root:category: 사회_사건
INFO:root:Article saved to ./donga/accident\2024-10-08-3.json
DEBUG:root:Scraping article: https://www.donga.com/news/Society/article/all/20241008/130173471/1
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.donga.com:443
DEBUG:urllib3.connectionpool:https://www.donga.com:443 "GET /news/Society/article/all/20241008/130173471/1 HTTP/11" 200 None
INFO:root:page_category: 사회


{'title': '‘밀양 성폭행’ 가해자들 신상 무단공개한 유튜버 구속', 'source_url': 'https://www.donga.com/news/Society/article/all/20241008/130173774/2', 'content': '2004년 발생한 ‘경남 밀양 집단 성폭력 사건’ 가해자들의 신상을 공개한 유튜브 채널 운영자가 구속됐다.\n경남경찰청은 정보통신망 이용촉진 및 정보보호 등에 관한 법률 위반(명예훼손)과 업무방해 등의 혐의로 유튜브 채널 운영자 20대 A 씨와 영상 제작자 30대 B 씨를 구속했다고 8일 밝혔다.\nA 씨 등은 올해 6~8월 자신의 유튜브 채널을 통해 밀양 집단 성폭행 사건 가해자 여러 명의 신상을 무단으로 공개한 혐의를 받는다.\n이들은 가해자 가족이 운영하고 있다며 특정 식당의 상호를 밝히기도 했다. 주로 사실이 확인되지 않은 제보나 인터넷 검색을 통해 자료를 수집한 것으로 알려졌다.\n경찰은 이 과정에서 밀양 성폭행 사건과 관련 없는 무고한 사람이 다수 포함됐다고 밝혔다. 그러면서 이들 외에도 비슷한 범죄를 저지른 다른 유튜버들에 대해서도 계속 수사 중이라고 전했다.', 'category': 'accident', 'source_created_at': '2024-10-08T10:33:00+09:00', 'press_id': 2, 'press_name': '동아일보'}


INFO:root:category: 사회_사건
INFO:root:Article saved to ./donga/accident\2024-10-08-4.json


{'title': '헤어진 연인의 보복성 소송에…法 “원고 청구 기각” 판결', 'source_url': 'https://www.donga.com/news/Society/article/all/20241008/130173471/1', 'content': '전 연인 생활비, 금전대여금 반환 청구\n대한법률구조공단은 헤어진 연인으로부터 부당한 대여금 및 물품반환 청구 소송에서 피고를 대리해 원고의 청구를 전부 기각시키는 성과를 냈다고 8일 밝혔다.\n공단에 따르면 대전지방법원 홍성지원 민사 1단독은 A씨가 과거 연인관계였던 B씨를 상대로 제기한 물품인도 등 청구소송에서 “원고의 청구를 모두 기각한다”고 판결했다.\nA씨는 옛 연인관계였던 B씨를 상대로 주택 매수자금, 생활비, 가게운영자금 등의 명목으로 금전을 대여했다고 주장하며 대여금 반환을 요구했다.\n또한 B씨가 자신의 소유 물품을 부당하게 점유하고 있다며 해당 물품의 인도를 요구하는 소송을 제기했다.\nB씨는 A씨의 주장을 전면 부인하며 금전 대여 사실이 없고, A씨의 물품을 점유한 사실도 없다며 반박했다.\n그러나 법무법인을 선임한 A씨의 청구에 속수무책으로 당할 수밖에 없는 상황이었다.\nA씨는 과거에도 B씨가 이별을 고하면 대여금 소송 및 형사 고소를 제기하기도 했고, 이 사건 소송 역시 보복성 소송의 성격이 강했다.\n공단은 B씨를 대리해 이 사건 청구소송에 응소했다.\n공단 측 변호사는 오히려 B씨가 A씨에게 도움을 줬지 A씨로부터 금전을 대여한 사실 자체가 없으며, 교제기간 상호간에 있지도 않은 금전 거래내역을 근거로 대여금 반환을 청구하고 있는 것이라며 적극 항변했다.\n법원은 공단의 항변을 받아들여 “원고의 청구를 모두 기각한다”고 판결했다.\n개인 간 금전거래 및 물품 소유권 분쟁에서 흔히 발생할 수 있는 입증의 어려움을 성공적으로 극복한 사례로 평가된다.\nA씨를 대리해 소송을 진행한 공단 소속 김상윤 변호사는 “법률에 대한 전문지식이 부족하고 경제적으로 어려운 일반 시민이 부당한 소송에 직

## 중앙일보

In [23]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import logging
import os
import re
from datetime import datetime
from urllib.parse import urlparse

logging.basicConfig(level='DEBUG')

header = {'User-Agent': ('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 '
                         '(KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'),}

output_dir = './joongang'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

categories = {
    '정치': 'https://www.joongang.co.kr/politics',
    '경제': 'https://www.joongang.co.kr/money',
    '국제': 'https://www.joongang.co.kr/world',
    '사회_사건': 'https://www.joongang.co.kr/society/accident', # 앞 분기 기준
    '경제_과학': 'https://www.joongang.co.kr/money/science', # 앞 분기 기준
    '사회': 'https://www.joongang.co.kr/society',
    '스포츠': 'https://www.joongang.co.kr/sports',
}


result_categories = {
    '정치': 'politics',
    '경제': 'economy',
    '국제': 'world',
    '사회_사건': 'accident',
    '경제_과학': 'science',
    '사회': 'society',
    '스포츠': 'sports',
}

# 현재 날짜의 오전 8시 (시간대는 +09:00 기준)
today = datetime.now().replace(hour=8, minute=0, second=0, microsecond=0, tzinfo=datetime.now().astimezone().tzinfo)

def scrap_article(article_url, category):
    response = requests.get(article_url, headers=header)
    soup = BeautifulSoup(response.text, 'html.parser')

    try:

        page_category = soup.find('a', {'class': 'title'}).get_text(strip=True)
        logging.info(f"page_category: {page_category}")
        
        logging.info(f"category: {category}")

        category_parts = category.split('_')
        
        # 카테고리가 아닌 글은 패스
        if page_category not in category_parts:
            logging.info(f"{article_url} 는 해당 카테고리가 아니어서 생략합니다. ")
            return None
    except AttributeError:
        logging.error(f"{article_url} 는 찾을 수 없습니다.")
        return None

    # 헤드라인 스크랩
    try:
        title = soup.find('h1', {'class': 'headline'}).get_text(strip=True)
    except AttributeError:
        title = 'N/A'

    # 생성일 스크랩
    try:
        date = soup.find('p', {'class': 'date'})
        time = date.find('time').get('datetime')
    except AttributeError:
        time = 'N/A'

    # 본문 스크랩
    try:
        content = soup.find('div', {'class': 'article_body'}).get_text(strip=True)
    except AttributeError:
        content = 'N/A'

    result_category = result_categories.get(category, 'unknown')

    return {'title': title, 'source_url': article_url, 'content': content, 'category': result_category, 'source_created_at': time, 'press_id': 1, 'press_name': '중앙일보'}

# 파일 생성
def save_article_as_json(article_data, date, file_idx):
    category = article_data.get('category')
    file_name = date + "-" + str(file_idx) + '.json'
    file_path = os.path.join(output_dir + "/" + category, file_name)
    
    # 단일 dataframe 생성
    single_article_df = pd.DataFrame([article_data])
    
    # json으로 변환
    single_article_df.to_json(file_path, orient='records', lines=True, force_ascii=False)
    logging.info(f"Article saved to {file_path}")

def check_for_new_articles(base_url, category):
    response = requests.get(base_url, headers=header)
    soup = BeautifulSoup(response.text, 'html.parser')

    article_count = 0
    cards = soup.find_all('li', class_='card')
    # cards.reverse()
    for card in cards:
        if article_count >= 4:
            break

        link_tag = card.find('a', href=True)
        if link_tag and 'href' in link_tag.attrs:
            article_link = link_tag['href']
        else:
            logging.warning("article_link is None or missing 'href'.")
            continue
    
        logging.debug(f"Scraping article: {article_link}")

        result = scrap_article(article_link, category)

        if result is None:
            continue

        if result.get('category') not in ['accident', 'science']:
            # 'source_created_at'를 datetime 객체로 변환
            article_time = datetime.strptime(result.get('source_created_at'), '%Y-%m-%dT%H:%M:%S%z')
            # 오늘 오전 8시 이후가 아니면 continue
            if article_time < today:
                logging.info(f"{article_link} 은 오늘 오전 8시 이전 기사이므로 생략합니다.")
                continue

        print(result)
        article_count += 1
        date = re.match(r'^[^T]+', result.get('source_created_at')).group()
        save_article_as_json(result, date, article_count)

def scrap_all_categories():
    
    for category, base_url in categories.items():
        logging.info(f"category (base_url): {category} ({base_url})")
        check_for_new_articles(base_url, category)
        time.sleep(2)

scrap_all_categories()

INFO:root:category (base_url): 사회_사건 (https://www.joongang.co.kr/society/accident)
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.joongang.co.kr:443
DEBUG:urllib3.connectionpool:https://www.joongang.co.kr:443 "GET /society/accident HTTP/11" 200 None
DEBUG:root:Scraping article: https://www.joongang.co.kr/article/25282740
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.joongang.co.kr:443
DEBUG:urllib3.connectionpool:https://www.joongang.co.kr:443 "GET /article/25282740 HTTP/11" 200 None
INFO:root:page_category: 여행레저
INFO:root:category: 사회_사건
INFO:root:https://www.joongang.co.kr/article/25282740 는 해당 카테고리가 아니어서 생략합니다. 
DEBUG:root:Scraping article: https://www.joongang.co.kr/article/25282789
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.joongang.co.kr:443
DEBUG:urllib3.connectionpool:https://www.joongang.co.kr:443 "GET /article/25282789 HTTP/11" 200 None
INFO:root:page_category: 사회
INFO:root:category: 사회_사건
INFO:root:Article s

{'title': '北에 보낸 나훈아·임영웅 영상…"오물풍선 취급당해" 국감 달군 이유', 'source_url': 'https://www.joongang.co.kr/article/25282789', 'content': '나훈아(왼쪽), 임영웅. 연합뉴스탈북민 단체가 북한에 날려 보낸 대형 풍선 안에 가수 나훈아, 임영웅의 공연 영상이 담긴 것을 두고 야당 의원이 저작권 문제를 제기했다. 앞서 정부도 "당사자 허락 없는 무단 복제, 배포는 저작권법 침해일 수 있다"고 밝힌 바 있다.7일 열린 국회 문화체육관광위원회 국정감사에서 강유정 더불어민주당 의원은 지난 6월 말 탈북민단체 자유북한운동연합이 북측에 국내 저작물을 보낸 것을 지적했다. 당시 이 단체는 북한의 오물풍선이 남측에 넘어온 것에 대한 대응으로 나훈아, 임영웅의 콘서트 영상과 노래를 담은 이동식 저장장치(USB) 5000개를 대형 풍선에 넣어 보냈다.강 의원은 "북한에서 우리에게 날아오는 걸 \'오물풍선\'이라고 부른다"면서 "(USB 담긴 풍선이) 북한에 날아가면 우리의 소중한 콘텐츠가 오물풍선 취급당하지 않겠냐"고 했다. 이어 "북한까지 못 가고 우리나라에 떨어진 것도 많이 확인됐기 때문에 문화체육관광부 소관이다. 수거해서 폐기해야 한다"고 말했다.유인촌 문체부 장관은 "불법 복제는 저작권 위반이고 예전부터 많이 수거하고 폐기도 했다"면서 "늘 하는 일이고 특별사법경찰도 운영하고 있다"고 답했다.이에 강 의원은 "문체부에 질의했더니 임영웅과 나훈아가 직접 대처해야 한다는 소극적 답변을 하고 있다"며 "허락을 안 받았으면 무조건 저작권 위반이다"고 주장했다.탈북민단체 자유북한운동연합이 6월 6일 오전 김정은 정권을 비난하는 전단 20만장과 나훈아·임영웅 트로트, 드라마 겨울연가 등을 저장한 USB 5000개, 1달러 지폐 2000장 등을 대형 풍선에 넣어 북한에 보냈다. 사진 박상학 대표 제공대북 풍선 관련 저작권 문제는 야권에서 지속적으로 제기하고 있는 문제다. 앞서 권칠승 민주당 의원도 문체부와

DEBUG:urllib3.connectionpool:https://www.joongang.co.kr:443 "GET /article/25282583 HTTP/11" 200 None
INFO:root:page_category: 정치
INFO:root:category: 사회_사건
INFO:root:https://www.joongang.co.kr/article/25282583 는 해당 카테고리가 아니어서 생략합니다. 
DEBUG:root:Scraping article: https://www.joongang.co.kr/article/25282633
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.joongang.co.kr:443
DEBUG:urllib3.connectionpool:https://www.joongang.co.kr:443 "GET /article/25282633 HTTP/11" 200 None
INFO:root:page_category: 라이프
INFO:root:category: 사회_사건
INFO:root:https://www.joongang.co.kr/article/25282633 는 해당 카테고리가 아니어서 생략합니다. 
DEBUG:root:Scraping article: https://www.joongang.co.kr/article/25282645
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.joongang.co.kr:443
DEBUG:urllib3.connectionpool:https://www.joongang.co.kr:443 "GET /article/25282645 HTTP/11" 200 None
INFO:root:page_category: 문화
INFO:root:category: 사회_사건
INFO:root:https://www.joongang.co.kr/article/25282645 는 해당

{'title': "피해자 개인정보 있는데…'박대성 살인사건 보고서' 온라인 유출", 'source_url': 'https://www.joongang.co.kr/article/25282657', 'content': '살인 혐의를 받는 박대성(30)이 지난 4일 오전 전남 순천경찰서에서 검찰로 송치되고 있다. 연합뉴스일면식 없는 10대 여고생을 살해한 박대성(30)의 범행 당일 경찰과 지자체가 작성한 상황 보고서가 온라인에 유출돼 경찰이 경위 파악에 나섰다.전남경찰청 사이버범죄수사대는 7일 박대성 살인사건 발생 보고서가 소셜미디어(SNS)를 중심으로 유포된 경위를 조사 중이라고 밝혔다.전남경찰청 강력계와 순천시 안전총괄과 등이 작성한 이 보고서는 대외 유출이 금지된 문서다. 피의자 박대성뿐만 아니라 피해자의 실명과 나이 등 개인정보가 담겨 있다.경찰은 사건 발생 직후인 다음날 오후 일반 시민으로부터 해당 내용을 신고받고 유출 사실을 인지했다. 전남경찰청 감찰부서에서 유출 사실을 인지한 뒤 사이버수사대에 수사의뢰했다.경찰은 입건 전 조사(내사)를 거쳐 최초 유포자를 특정하면 공무상비밀누설 또는 개인정보보호법 위반 혐의로 전환할 계획이다.박대성은 지난달 26일 0시44분께 전남 순천시 조례동 한 길거리에서 A양(18)을 흉기로 찌르고 달아난 혐의(살인)로 구속 송치됐다.A양은 크게 다쳐 병원으로 이송됐으나 숨졌다. A양은 몸이 불편한 아버지의 약을 사러 나갔다가 친구를 만나고 귀가하는 길에 피살됐다.박대성은 배달음식점을 운영하는 가게에서 홀로 술을 마시다가 흉기를 챙겨 밖으로 나왔고 일면식 없는 A양을 800m가량 쫓아가 범행한 것으로 조사됐다.경찰은 수단의 잔인성·국민의 알권리·중대한 피해 등을 고려해 박대성의 신상·머그샷 얼굴 사진을 지난달 30일 공개했다.한영혜 기자 han.younghye@joongang.co.kr', 'category': 'accident', 'source_created_at': '2024-10-08T00:00:55+09:00', 'pres

INFO:root:page_category: 사회
INFO:root:category: 사회_사건
INFO:root:Article saved to ./joongang/accident\2024-10-07-3.json
DEBUG:root:Scraping article: https://www.joongang.co.kr/article/25282511
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.joongang.co.kr:443


...

{'title': '[Biz & Now] SK AI 서밋 내달 4일 개막, 전세계 석학 참가', 'source_url': 'https://www.joongang.co.kr/article/25282685', 'content': '국내·외 저명한 인공지능(AI) 분야 인사들이 참석하는 행사가 서울에서 열린다. SK텔레콤은 다음달 4~5일에 서울 강남구 코엑스에서 ‘SK AI 서밋 2024’를 연다고 7일 밝혔다. 생성 AI 시대 대표주자인 오픈AI의 그레그 브로크먼 회장 겸 사장이 참석한다. 브로크먼 사장은 직접 무대에 올라 ‘AI의 미래 비전’이라는 주제로 좌담에 참여할 예정이다. 라니 보카르 마이크로소프트(MS) 총괄 부사장, 리 카이푸 시노베이션 벤처스 및 라임즈AI 회장 등도 참석한다.', 'category': 'science', 'source_created_at': '2024-10-08T00:02:02+09:00', 'press_id': 1, 'press_name': '중앙일보'}


INFO:root:page_category: 경제
INFO:root:category: 경제_과학
INFO:root:Article saved to ./joongang/science\2024-10-08-3.json
DEBUG:root:Scraping article: https://www.joongang.co.kr/article/25282664
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.joongang.co.kr:443
DEBUG:urllib3.connectionpool:https://www.joongang.co.kr:443 "GET /article/25282664 HTTP/11" 200 None


{'title': '[Biz & Now] 정부 “틱톡 정보통신망법 위반 여부 조사”', 'source_url': 'https://www.joongang.co.kr/article/25282678', 'content': '7일 정보통신업계와 관계 당국 등에 따르면 방송통신위원회는 중국 숏폼(짧은 영상) 앱 틱톡이 정보통신망법을 위반했다고 보고 조만간 한국인터넷진흥원(KISA)을 통해 조사에 착수할 계획이다. 틱톡은 마케팅·광고 수신 동의에 해당하는 부분을 개인정보 처리방침 동의 항목에 묶인 ‘필수 동의’ 대신 ‘선택 동의’로 해야 하지만 이용자 가입 즉시 강제로 광고 동의가 이뤄져 문제시되고 있다.', 'category': 'science', 'source_created_at': '2024-10-08T00:02:01+09:00', 'press_id': 1, 'press_name': '중앙일보'}


INFO:root:page_category: 경제
INFO:root:category: 경제_과학
INFO:root:Article saved to ./joongang/science\2024-10-08-4.json


{'title': '노벨생리의학상에 ‘마이크로 RNA 발견’ 미국인 듀오', 'source_url': 'https://www.joongang.co.kr/article/25282664', 'content': '빅터 앰브로스(左), 게리 러브컨(右)암과 같은 난치병 차세대 치료제로 주목받고 있는 ‘마이크로 RNA(miRNA)’를 발견한 연구자들이 올해 노벨 생리의학상을 받았다.스웨덴 카롤린스카의대 노벨위원회는 2024년 노벨 생리의학상 수상자로 빅터 앰브로스 미국 매사추세츠 의대 교수와 게리 러브컨 하버드대 의대 교수를 공동 선정했다고 7일(현지시간) 발표했다. 위원회는 이들이 유전자에 의해 생명체를 구성하는 단백질을 만들어내는 과정(유전체 발현)을 조절하는 마이크로 RNA를 처음 발견하고 그 역할을 규명한 공로를 인정했다.앰브로스 교수는 미생물인 ‘예쁜꼬마선충’을 연구하는 과정에서 처음 마이크로 RNA를 발견했다. 러브컨 교수는 이후 마이크로 RNA가 생물의 유전체 발현에 어떤 역할을 하는지 밝혀냈다. 위원회는 “(마이크로 RNA의 발견은) 유전자 조절과 관련된 연구에 새로운 차원을 열어줬다”며 “마이크로 RNA는 유기체의 발달과 기능에 근본적으로 중요한 역할을 하는 것으로 입증되고 있다”고 밝혔다. 두 수상자는 상금 1100만 스웨덴 크로나(약 14억3000만원)를 나눠 받는다.마이크로 RNA는 20~24개의 염기로 이뤄진 작은 ‘리보핵산(RNA)’으로, 세포 내에 유전체 발현을 미세하게 조절하는 중요한 분자다. 유전 정보를 담고 있는 DNA는 세포질 속 단백질을 만들어내는 기관으로 유전 정보를 전달하는 ‘메신저 RNA(mRNA)’를 만들어내는데, 마이크로 RNA는 mRNA가 각 세포에 맞게 적절한 양의 단백질을 만들어낼 수 있게 조절해주는 역할을 한다.장수환 울산의대 서울아산병원 생리학교실 교수는 “마이크로 RNA는 세포 성장, 발달, 분화 등 여러 중요한 생물학적 과정에서 필수적인 역할을 한다”며 “유전체 발현 이상에 의해 발생하는 다양한 질환의 치료를 위

## 한국경제

In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import logging
from urllib.parse import urlparse

logging.basicConfig(level='DEBUG')

header = {'User-Agent': ('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 '
                         '(KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'),}

categories = {
    '경제': 'https://www.hankyung.com/economy',
    '정치': 'https://www.hankyung.com/politics'
}


def scrap_article(article_url, category):
    response = requests.get(article_url, headers=header)
    soup = BeautifulSoup(response.text, 'html.parser')

    try:
      
        # 해당 페이지 카테고리
        page_category = soup.find('a', {'class': 'title'}).get_text(strip=True)
        logging.info(f"page_category: {page_category}")
        
        # 찾고있는 카테고리
        logging.info(f"category: {category}")

        category = category.split('_')
        
        # 페이지 카테고리 != 찾고있는 카테고리 -> 스킵
        if page_category not in category:
            logging.info(f"{article_url} 는 해당 카테고리가 아니어서 생략합니다. ")
            return None
        
    except AttributeError:
        # 유효하지 않은 url 스킵
        logging.error(f"{article_url} 는 찾을 수 없습니다.")
        return None

    # 헤드라인 스크랩
    try:
        headline = soup.find('h3', {'class': 'next-fit'}).get_text(strip=True)
    except AttributeError:
        headline = 'N/A'

    # 생성일 스크랩
    try:
        date = soup.find('span', {'class': 'txt-date'})
        time = date.find('time').get('datetime')
    except AttributeError:
        time = 'N/A'

    # 본문 스크랩
    try:
        content = soup.find('p', {'class': 'lead'}).get_text(strip=True)
    except AttributeError:
        content = 'N/A'

    
    return {'title': headline, 'time': time, 'content': content}


def check_for_new_articles(base_url, category):
    
    # 결과 받아오기
    response = requests.get(base_url, headers=header, allow_redirects=False)
    print(f"Status Code: {response.status_code}")
    print(f"Location (Redirect URL): {response.headers.get('Location')}")
    
    # 받은 결과를 BeautifulSoup로 전달
    soup = BeautifulSoup(response.text, 'html.parser')

    # 현재 가져온 기사 갯수 초기화
    article_count = 0
    
    # 해당 태그와 일치하는 내용 가져오기
    articles = soup.find_all('h3', class_='news-fit')
    
    # 최신 기사를 가져오기 위하 정렬 수행
    articles.reverse()
    
    # 가져온 card를 탐색
    for article in articles:
        
        # 기사를 3개 추출한 경우 종료
        if article_count >= 3:
            break

        # 기사 url 추출
        link_tag = article.find('a', href=True)
        if link_tag and 'href' in link_tag.attrs:
            article_link = link_tag['href']
        else:
            logging.warning("article_link is None or missing 'href'.")
            continue
    
        logging.debug(f"Scraping article: {article_link}")

        # 기사 추출
        result = scrap_article(article_link, category)

        # 추출된 기사가 있는 경우에만 다음 로직을 수행
        if result:

            print(result)
            # 카운팅
            article_count += 1
    

def scrap_all_categories():
    
    for category, base_url in categories.items():
        logging.info(f"category (base_url): {category} ({base_url})")
        check_for_new_articles(base_url, category)
        time.sleep(2)

scrap_all_categories()

INFO:root:category (base_url): 경제 (https://www.hankyung.com/economy)
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.hankyung.com:443
DEBUG:urllib3.connectionpool:https://www.hankyung.com:443 "GET /economy HTTP/11" 200 None


Status Code: 200
Location (Redirect URL): None


INFO:root:category (base_url): 정치 (https://www.hankyung.com/politics)
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.hankyung.com:443
DEBUG:urllib3.connectionpool:https://www.hankyung.com:443 "GET /politics HTTP/11" 200 None


Status Code: 200
Location (Redirect URL): None
